<a href="https://colab.research.google.com/github/john-d-noble/callcenter/blob/main/2_CB_Step_6_synthetic_Call_Volume.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Cleaned Code for Data Preparation

This section contains the complete and cleaned code to load and prepare all datasets, generate synthetic data, combine everything into a single DataFrame, and save the final result to `all_combined_data.csv`.

In [17]:
# --- 1. Load Original Data and Prepare for Combination ---
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime

# Load the original data
df_original = pd.read_csv('agent_contact_volume_wgsd2.csv')

# Convert the date column to datetime objects
df_original['V Cx Contact Volume Template Created Datetime Utc Date'] = pd.to_datetime(df_original['V Cx Contact Volume Template Created Datetime Utc Date'])

# Rename the contact volume column for consistency
df_original = df_original.rename(columns={'V Cx Contact Volume Template Contacts': 'Calls'})

# Set the date column as the index for merging
df_original = df_original.set_index('V Cx Contact Volume Template Created Datetime Utc Date')

# Keep only the 'Calls' column from the original data for concatenation
df_original_calls = df_original[['Calls']]

display(df_original_calls.head())
display(df_original_calls.tail())

,Calls
V Cx Contact Volume Template Created Datetime Utc Date,
2023-01-01,2882
2023-01-02,5055
2023-01-03,6537
2023-01-04,7238
2023-01-05,7302


,Calls
V Cx Contact Volume Template Created Datetime Utc Date,
2025-08-31,4601
2025-09-01,6793
2025-09-02,8868
2025-09-03,9748
2025-09-04,2136


In [18]:
# --- 2. Generate Synthetic Data ---

# Calculate day of week statistics from the original data
df_original['DayOfWeek_Original'] = df_original.index.dayofweek # Use a temporary name to avoid conflict
day_of_week_stats = df_original.groupby('DayOfWeek_Original')['Calls'].agg(['mean', 'std']).to_dict()

start_date_synthetic = '2021-01-01'
end_date_synthetic = '2022-12-31'
date_rng_synthetic = pd.date_range(start=start_date_synthetic, end=end_date_synthetic, freq='D')
df_synthetic = pd.DataFrame(date_rng_synthetic, columns=['Date'])
df_synthetic['DayOfWeek_Synthetic'] = df_synthetic['Date'].dt.dayofweek # Use a temporary name

def generate_calls(row):
    day = row['DayOfWeek_Synthetic']
    # Handle cases where a day of week might not be in the original data (though unlikely with this dataset)
    if day in day_of_week_stats['mean'] and day in day_of_week_stats['std']:
        mean = day_of_week_stats['mean'][day]
        std_dev = day_of_week_stats['std'][day]
        noise = np.random.normal(0, std_dev / 2) # Add realistic variation
        return max(0, int(mean + noise))
    else:
        # Return a default or handle missing stats as appropriate
        return 0

# Apply generate_calls row-wise
df_synthetic['Calls'] = df_synthetic.apply(generate_calls, axis=1)


# Set the Date column as the index for concatenation
df_synthetic = df_synthetic.set_index('Date')

# Keep only the 'Calls' column from the synthetic data
df_synthetic_calls = df_synthetic[['Calls']]

display(df_synthetic_calls.head())
display(df_synthetic_calls.tail())

,Calls
Date,
2021-01-01,8715
2021-01-02,6005
2021-01-03,5340
2021-01-04,9364
2021-01-05,8726


,Calls
Date,
2022-12-27,11252
2022-12-28,8739
2022-12-29,9159
2022-12-30,9151
2022-12-31,5547


In [24]:
# --- 3. Load Market Data ---

# Parameters
start_date_market = '2020-12-31'
end_date_market = datetime.now().strftime('%Y-%m-%d')
tickers = {
    '^VIX': '^VIX',
    'SPY': 'SPY',
    'QQQ': 'QQQ',
    'DX-Y.NYB': 'DX-Y.NYB',
    'GC=F': 'GC=F',
    'BTC-USD': 'BTC-USD', # Added BTC-USD
    'ETH-USD': 'ETH-USD'  # Added ETH-USD
}

# Download full market data
market_data = pd.DataFrame()
# Use the full date range from the start of market data to the end of original data for reindexing
full_date_range = pd.date_range(start=start_date_market, end=df_original_calls.index.max())


# Define known launch dates (approximate based on common knowledge or initial data inspection)
launch_dates = {
    'CVOL-USD': pd.to_datetime('2022-02-28'),
    'CVX-USD': pd.to_datetime('2021-05-17'),
    # Add other tickers and their launch dates if necessary
}

selected_columns = {
    '^VIX': ['Close'],
    'SPY': ['Close', 'Volume'],
    'QQQ': ['Close', 'Volume'],
    'DX-Y.NYB': ['Close'],
    'GC=F': ['Close', 'Volume'],
    'BTC-USD': ['Close', 'Volume'],
    'ETH-USD': ['Close', 'Volume']
}


for label, ticker in tickers.items():
    data = yf.download(ticker, start=start_date_market, end=end_date_market)
    # Select only the specified columns
    if label in selected_columns:
        data = data[selected_columns[label]]
    else:
        # Default to selecting all columns if not specified
        data = data[['Open', 'High', 'Low', 'Close', 'Volume']]

    # Reindex to the full date range based on the combined data
    data = data.reindex(full_date_range)

    # Handle pre-launch dates by setting to 0
    if label in launch_dates:
        pre_launch_dates = data.index < launch_dates[label]
        data.loc[pre_launch_dates] = 0 # Use .loc for setting values

    # Forward-fill gaps after launch
    data = data.ffill()

    data.columns = [f"{col}_{label}" for col in data.columns] # Rename columns
    if market_data.empty:
        market_data = data
    else:
        market_data = market_data.join(data, how='outer')

# Display the head of the downloaded market data
display(market_data.head())
display(market_data.tail())

/tmp/ipython-input-1122018128.py:41: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date_market, end=end_date_market)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1122018128.py:41: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date_market, end=end_date_market)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1122018128.py:41: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date_market, end=end_date_market)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1122018128.py:41: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date_market, end=end_date_market)
[*********************100%*******

,"('Close', '^VIX')_^VIX","('Close', 'SPY')_SPY","('Volume', 'SPY')_SPY","('Close', 'QQQ')_QQQ","('Volume', 'QQQ')_QQQ","('Close', 'DX-Y.NYB')_DX-Y.NYB","('Close', 'GC=F')_GC=F","('Volume', 'GC=F')_GC=F","('Close', 'BTC-USD')_BTC-USD","('Volume', 'BTC-USD')_BTC-USD","('Close', 'ETH-USD')_ETH-USD","('Volume', 'ETH-USD')_ETH-USD"
2020-12-31,22.750000,351.009918,78520700.0,305.207794,21611400.0,89.940002,1893.099976,127.0,29001.720703,46754964848,737.803406,13926846861
2021-01-01,22.750000,351.009918,78520700.0,305.207794,21611400.0,89.940002,1893.099976,127.0,29374.152344,40730301359,730.367554,13652004358
2021-01-02,22.750000,351.009918,78520700.0,305.207794,21611400.0,89.940002,1893.099976,127.0,32127.267578,67865420765,774.534973,19740771179
2021-01-03,22.750000,351.009918,78520700.0,305.207794,21611400.0,89.940002,1893.099976,127.0,32782.023438,78665235202,975.507690,45200463368
2021-01-04,26.969999,346.231293,110210800.0,300.898285,45305900.0,89.879997,1944.699951,154.0,31971.914062,81163475344,1040.233032,56945985763


,"('Close', '^VIX')_^VIX","('Close', 'SPY')_SPY","('Volume', 'SPY')_SPY","('Close', 'QQQ')_QQQ","('Volume', 'QQQ')_QQQ","('Close', 'DX-Y.NYB')_DX-Y.NYB","('Close', 'GC=F')_GC=F","('Volume', 'GC=F')_GC=F","('Close', 'BTC-USD')_BTC-USD","('Volume', 'BTC-USD')_BTC-USD","('Close', 'ETH-USD')_ETH-USD","('Volume', 'ETH-USD')_ETH-USD"
2025-08-31,15.36,645.049988,74522200.0,570.400024,56030400.0,97.769997,3473.699951,0.0,108236.710938,47986191770,4390.019043,26683044984
2025-09-01,15.36,645.049988,74522200.0,570.400024,56030400.0,97.769997,3473.699951,0.0,109250.593750,66870372995,4314.470215,37530746508
2025-09-02,17.17,640.270020,81983500.0,565.619995,65876800.0,98.400002,3549.399902,658.0,111200.585938,74776999491,4325.365723,39884692334
2025-09-03,16.35,643.739990,70820900.0,570.070007,54230200.0,98.139999,3593.199951,72.0,111723.210938,61119643565,4450.389160,35260873497
2025-09-04,15.30,649.119995,65219200.0,575.229980,47526300.0,98.349998,3565.800049,237.0,110723.601562,60131132901,4298.744141,34919798552


In [25]:
# --- 4. Combine Call Volume Data (Synthetic and Original) ---

# Concatenate the synthetic and original call volume dataframes
# The index (Date) will be used for alignment
df_combined_calls = pd.concat([df_synthetic_calls, df_original_calls])

# Sort the combined dataframe by date to ensure correct time series order
df_combined_calls = df_combined_calls.sort_index()

display(df_combined_calls.head())
display(df_combined_calls.tail())

,Calls
2021-01-01,8715
2021-01-02,6005
2021-01-03,5340
2021-01-04,9364
2021-01-05,8726


,Calls
2025-08-31,4601
2025-09-01,6793
2025-09-02,8868
2025-09-03,9748
2025-09-04,2136


In [26]:
# --- 5. Merge Combined Call Volume with Market Data and Finalize ---

# Merge the combined call volume dataframe with the market data dataframe on their index (Date)
df_all_combined_cleaned = df_combined_calls.join(market_data, how='left')

# Add a single, fully populated 'DayOfWeek' column derived from the index (Date)
df_all_combined_cleaned['DayOfWeek'] = df_all_combined_cleaned.index.dayofweek

# Display the head of the final combined and cleaned DataFrame
display(df_all_combined_cleaned.head())
display(df_all_combined_cleaned.tail())

# Save the final combined DataFrame to a new CSV file
df_all_combined_cleaned.to_csv('all_combined_data.csv')

,Calls,"('Close', '^VIX')_^VIX","('Close', 'SPY')_SPY","('Volume', 'SPY')_SPY","('Close', 'QQQ')_QQQ","('Volume', 'QQQ')_QQQ","('Close', 'DX-Y.NYB')_DX-Y.NYB","('Close', 'GC=F')_GC=F","('Volume', 'GC=F')_GC=F","('Close', 'BTC-USD')_BTC-USD","('Volume', 'BTC-USD')_BTC-USD","('Close', 'ETH-USD')_ETH-USD","('Volume', 'ETH-USD')_ETH-USD",DayOfWeek
2021-01-01,8715,22.750000,351.009918,78520700.0,305.207794,21611400.0,89.940002,1893.099976,127.0,29374.152344,40730301359,730.367554,13652004358,4
2021-01-02,6005,22.750000,351.009918,78520700.0,305.207794,21611400.0,89.940002,1893.099976,127.0,32127.267578,67865420765,774.534973,19740771179,5
2021-01-03,5340,22.750000,351.009918,78520700.0,305.207794,21611400.0,89.940002,1893.099976,127.0,32782.023438,78665235202,975.507690,45200463368,6
2021-01-04,9364,26.969999,346.231293,110210800.0,300.898285,45305900.0,89.879997,1944.699951,154.0,31971.914062,81163475344,1040.233032,56945985763,0
2021-01-05,8726,25.340000,348.615875,66426200.0,303.378876,29323400.0,89.440002,1952.699951,113.0,33992.429688,67547324782,1100.006104,41535932781,1


,Calls,"('Close', '^VIX')_^VIX","('Close', 'SPY')_SPY","('Volume', 'SPY')_SPY","('Close', 'QQQ')_QQQ","('Volume', 'QQQ')_QQQ","('Close', 'DX-Y.NYB')_DX-Y.NYB","('Close', 'GC=F')_GC=F","('Volume', 'GC=F')_GC=F","('Close', 'BTC-USD')_BTC-USD","('Volume', 'BTC-USD')_BTC-USD","('Close', 'ETH-USD')_ETH-USD","('Volume', 'ETH-USD')_ETH-USD",DayOfWeek
2025-08-31,4601,15.36,645.049988,74522200.0,570.400024,56030400.0,97.769997,3473.699951,0.0,108236.710938,47986191770,4390.019043,26683044984,6
2025-09-01,6793,15.36,645.049988,74522200.0,570.400024,56030400.0,97.769997,3473.699951,0.0,109250.593750,66870372995,4314.470215,37530746508,0
2025-09-02,8868,17.17,640.270020,81983500.0,565.619995,65876800.0,98.400002,3549.399902,658.0,111200.585938,74776999491,4325.365723,39884692334,1
2025-09-03,9748,16.35,643.739990,70820900.0,570.070007,54230200.0,98.139999,3593.199951,72.0,111723.210938,61119643565,4450.389160,35260873497,2
2025-09-04,2136,15.30,649.119995,65219200.0,575.229980,47526300.0,98.349998,3565.800049,237.0,110723.601562,60131132901,4298.744141,34919798552,3
